<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/main/Yurkov%20Vladimir/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparations

Импортируем модули

In [1]:
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

Загружаем данные и преобразуем их в pandas dataframe

In [4]:
url = 'https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%92%D0%B5%D1%81%D1%8C%20%D0%BF%D0%B5%D1%80%D0%B8%D0%BE%D0%B4.csv'
file_name = 'data.csv'
gdown.download(url, file_name, quiet=False)
df_backup = pd.read_csv(file_name, sep = ';', on_bad_lines='skip', engine='python' )
df_backup.head()

Downloading...
From: https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%92%D0%B5%D1%81%D1%8C%20%D0%BF%D0%B5%D1%80%D0%B8%D0%BE%D0%B4.csv
To: /content/data.csv
100%|██████████| 23.0M/23.0M [00:00<00:00, 34.9MB/s]


,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,ХирургНеИсп3_Диагноз_ЗначениеПредставлениеПредставление,ХирургНеИсп3_МКБ101,ХирургНеИсп3_МКБ102,ХирургНеИсп3_МКБ103,ЭКГНеИсп1_МКБ101,ЭКГНеИсп1_МКБ102,ЭКГНеИсп2_МКБ101,ЭКГНеИсп2_МКБ102,Эндоскопия1_МКБ101,Эндоскопия1_МКБ102
0,79292387-652a-11eb-80e7-0cc47aab8067,dca98d4e-2098-11ea-80d6-0cc47aab8067,1989-12-31T00:00:00,Женский,Женский,2021-02-02T00:00:00,Периодический,Периодический,False,3d01b2d6-ae08-11e9-80cf-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78efe744-9c36-11eb-80eb-0cc47aab8067,b35fb2bb-9c35-11eb-80eb-0cc47aab8067,1998-07-20T00:00:00,Женский,Женский,2021-04-13T00:00:00,Периодический,Периодический,False,3d01b2d6-ae08-11e9-80cf-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,131f68ef-9c2d-11eb-80eb-0cc47aab8067,f8c98e39-9c2c-11eb-80eb-0cc47aab8067,2002-01-04T00:00:00,Женский,Женский,2021-04-13T00:00:00,Периодический,Периодический,False,3d01b2d6-ae08-11e9-80cf-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,941e4b56-9c2d-11eb-80eb-0cc47aab8067,7ea364a1-9c2d-11eb-80eb-0cc47aab8067,1991-08-26T00:00:00,Женский,Женский,2021-04-13T00:00:00,Периодический,Периодический,False,3d01b2d6-ae08-11e9-80cf-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5a24806c-f033-11eb-80f6-0cc47aab8067,52eb6a7d-c1ab-11ea-80dd-0cc47aab8067,1999-03-21T00:00:00,Женский,Женский,2021-07-29T00:00:00,Периодический,Периодический,False,3d01b2d6-ae08-11e9-80cf-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Getting started with a dataframe

Делаем копию датафрейма

In [5]:
df = df_backup.copy()

Блок с функциями.

* `clean_clear` - функция, удаляющая дубликаты строк/столбцов и полностью пустые
строки/столбцы.
* `regex_to_nan` - функция, находящая значения по регулярному выражению и заменяющая его на `np.nan`.
* `fills_count` - функция, возвращающая заполненность датафрейма в процентах.

In [6]:
def clean_clear(df, pflag=False):
    # Сохраняем размерность датафрейма
    shape_sum = df.shape

    # Удаляем дубликаты строк
    df = df.drop_duplicates()
    # Удаляем дубликаты столбцов
    df = df.T.drop_duplicates().T

    # Удаляем полностью пустые столбцы и строки
    df = df.dropna(axis=0, how='all')
    df = df.dropna(axis=1, how='all')

    # Выводим результаты
    if pflag:
        print(f"Форма датасета до операции: {shape_sum}")
        print(f"Форма датасета после операции: {df.shape}")

        print(f"Количество строк после удаления пустых и повторяющихся: {df.shape[0]}")
        print(f"Количество столбцов после удаления пустых и повторяющихся: {df.shape[1]}")
        print(f"\n*Удалено {shape_sum[0] - df.shape[0]} строк и {shape_sum[1] - df.shape[1]} столбцов.")
    df = df.convert_dtypes()
    return df

# Регулярное выражения для GUID
regex_GUID = "^[{]?[0-9a-fA-F]{8}" + "-([0-9a-fA-F]{4}-)" + "{3}[0-9a-fA-F]{12}[}]?$"

# Поиск и удаление GUID столбцов
def regex_to_nan(df, to_replace=regex_GUID, value=np.nan, pflag=False):
    df = df.replace(to_replace=to_replace, value=value, regex=True)
    if pflag:
        print(f"Колонки для удаления:\n{df.columns[df.isna().all()].tolist()}")
    return df

# Подсчет заполненности таблицы
def fills_count(df):
    return f"Заполненность: ~{round((1 - (df.isna().sum().sum() / df.size))*100, 2)}%"

Анализ данных.
* форма
* количество значений
* основная информация о датафрейме
* определение и подсчет типов данных до и после попытки автоматического преобразования типов `df.convert_dtypes()`
* процент заполненности данными

In [7]:
print()
print("Форма: ", df.shape)
print("Количество значений: ", df.size)
print()
display(df.info(max_cols=0))
# pd.set_option('display.max_rows', None) # (default=20) для сброса pd.reset_option('display.max_rows') | для колонок max_columns
# display(df.dtypes)
# pd.reset_option('display.max_rows')
print("\nТипы данных в фрейме:")
display(df.dtypes.value_counts())
print("\nПреобразование типов данных.")
df = df.convert_dtypes()
print("\nТипы данных после преобразования:")
display(df.dtypes.value_counts())
print()
print(fills_count(df))


Форма:  (32686, 289)
Количество значений:  9446254

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Columns: 289 entries, ДокументПрохождениеМедосмотра to Эндоскопия1_МКБ102
dtypes: bool(1), float64(75), object(213)
memory usage: 71.9+ MB


None


Типы данных в фрейме:


object     213
float64     75
bool         1
dtype: int64


Преобразование типов данных.

Типы данных после преобразования:


string     213
Int64       75
boolean      1
dtype: int64


Заполненность: ~6.93%


Поиск колонок с датами.
* нахождение по обнаруженному типу `datetime64[ns]`
* нахождение колонок, содержащих в названии слово `Дата`

Сопоставление полученных результатов между собой.
Получилось, что данные по слову `Дата` имеют действительно даты, только не определились и полностью включают в себя найденные стобцы по автоматически определенному типу данных - `datetime64[ns]`

In [8]:
datatype_list = df.columns[df.dtypes == 'datetime64[ns]'].values.tolist()
print(f"Количество столбцов с типом 'datetime64[ns]': {len(datatype_list)}")
dataword_list = df.filter(regex=re.compile("дата", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Количество столбцов со словом 'Дата': {len(dataword_list)}")
print()
if len(datatype_list) > len(dataword_list):
    data_diff = set(datatype_list) - set(dataword_list)
    if set(dataword_list).issubset(datatype_list):
        print("Список dataword_list входит в datatype_list\n")
elif len(datatype_list) < len(dataword_list):
    data_diff =  set(dataword_list) - set(datatype_list)
    if set(datatype_list).issubset(dataword_list):
        print("Список datatype_list входит в dataword_list\n")
else: print("Списки равны\n")

# Для вывода списка разницы колонок убрать комментарий с строки
# print(*data_diff, sep="\n")

Количество столбцов с типом 'datetime64[ns]': 0
Количество столбцов со словом 'Дата': 2

Список datatype_list входит в dataword_list



Ищем колонки с Ф.И.О. врачей и названием медицинского учреждения

In [9]:
datadoctor_list = df.filter(regex=re.compile("врач", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Колчество колонок с Ф.И.О. врачей: {len(datadoctor_list)}")
datasalon_list = df.filter(regex=re.compile("салон", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Колчество колонок с названием мед. учреждения: {len(datasalon_list)}")
dataspec_list = df.filter(regex=re.compile("специализация", re.IGNORECASE), axis=1).columns.values.tolist()
print(f"Колчество колонок с названием специализации: {len(datasalon_list)}")

Колчество колонок с Ф.И.О. врачей: 0
Колчество колонок с названием мед. учреждения: 0
Колчество колонок с названием специализации: 0


Анализ и проверка целевой колонки `ЗаключениеМК` на предмет "мусорных" данных и пропусков.

In [10]:
display(df['ЗаключениеМК'].unique().tolist())

['Годен',
 'ГоденСКоррекциейЗрения',
 'НуждаетсяВДообследованииИЛечении',
 <NA>,
 'ГоденБезРаботНаВысотах',
 'ГоденСКоррекциейСлуха',
 ' п. 11.3',
 'ОграниченноГоден',
 'машинист бульдозера',
 'Газоспасатель',
 ' Газоспасатель',
 'Негоден']

Создание маски с соответствующими критериям данными для целевой колонки.

In [11]:
mask_list = ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах',
        'ГоденСКоррекциейСлуха', 'НуждаетсяВДообследованииИЛечении',
        'ВременноНегоден', 'ОграниченноГоден', 'Негоден']

Чистка от мусорных, дублирующих, не имеющих ценности и пустых данных.

Сначала производится чистка по маске из целевой колонки `ЗаключениеМК`, затем удаляются данные с датами, затем данные с GUID, дублирующие и пустые.

Сравнивается форма датафрейма до и после чистки.

In [12]:
print(f"Форма датафрейма до: {df.shape}")
try:
    print("Чистка от некорректных данных в целевой колонке")
    df = df[df['ЗаключениеМК'].isin(mask_list)]
    print(df.shape)

    print("Чистка от колонок с датами")
    df = df.drop(dataword_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с Ф.И.О. врачей")
    df = df.drop(datadoctor_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с названием мед. учреждений")
    df = df.drop(datasalon_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с названием специализации врача")
    df = df.drop(dataspec_list, axis='columns')
    print(df.shape)

    print("Чистка от колонок с GUID, полностью пустых колонок и строк, дубликатов колонок и строк...")
    df = regex_to_nan(df)
    df = clean_clear(df)
except Exception as e:
    print(e)
print(f"Форма датафрейма после: {df.shape}")

Форма датафрейма до: (32686, 289)
Чистка от некорректных данных в целевой колонке
(19200, 289)
Чистка от колонок с датами
(19200, 287)
Чистка от колонок с Ф.И.О. врачей
(19200, 287)
Чистка от колонок с названием мед. учреждений
(19200, 287)
Чистка от колонок с названием специализации врача
(19200, 287)
Чистка от колонок с GUID, полностью пустых колонок и строк, дубликатов колонок и строк...
Форма датафрейма после: (12370, 138)


#Categorization, OHE, Tokenization, BoW, etc...

Проверим типы данных в датафрейме

In [14]:
display(df.dtypes.value_counts())

string     137
boolean      1
dtype: int64

In [15]:
display(df.head())

,КлиентПол,КлиентПолПредставление,ТипМедосмотра,ПсихОсвидетельствование,Цех,Профессия,ЗаключениеМК,ЗаключениеМКПредставление,Заключение,ВредныеФакторы,...,Хирургия3_МКБ101,Хирургия3_МКБ102,ХирургНеИсп1_МКБ101,ХирургНеИсп1_МКБ102,ХирургНеИсп1_МКБ103,ХирургНеИсп2_МКБ101,ЭКГНеИсп1_МКБ101,ЭКГНеИсп1_МКБ102,Эндоскопия1_МКБ101,Эндоскопия1_МКБ102
0,Женский,Женский,Периодический,False,<NA>,Продавец-кассир,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Женский,Женский,Периодический,False,<NA>,Продавец-кассир,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,04.окт,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Женский,Женский,Периодический,False,<NA>,Младший продавец-консультант,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Женский,Женский,Периодический,False,<NA>,Менеджер магазина,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Женский,Женский,Периодический,False,<NA>,Продавец-кассир,Годен,Годен к работе в указанных условиях,Годен к работе в указанных условиях,23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
